In [1]:
from __future__ import division
#import numpy as np
import pickle
import rospy
import sys
from geometry_msgs.msg import Twist
from sensor_msgs.msg import Imu, JointState
from std_srvs.srv import Empty
from nav_msgs.msg import Odometry
from sensor_msgs.msg import LaserScan
from std_msgs.msg import Float32, Float64, Int32
import time
import pandas as pd
from matplotlib import pyplot as plt
from collections import deque
import random
import tensorflow as tf
import numpy as np

import torch
import torch.nn as nn
from torch.distributions import MultivariateNormal
import gym
import numpy as np

from gazebo_msgs.msg import ModelStates
from tensorboardX import SummaryWriter
import math

/home/giuseppe/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/giuseppe/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/giuseppe/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/giuseppe/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.

In [2]:
writer = SummaryWriter("logdir")

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
wheel_l = "/cmd_vel"#/self_balancing_ai/wheell_velocity_controller/command"
wheel_r = "/cmd_vel"#/self_balancing_ai/wheelr_velocity_controller/command"
real_robot = "/real_robot_wheel"


class NN_ROBOT1:
    def __init__(self):
        
        self.joint1 = rospy.Publisher(wheel_l,Twist,queue_size =1)
        self.joint2 = rospy.Publisher(wheel_r,Twist,queue_size =1)      
        self.real_robot = rospy.Publisher(real_robot,Int32,queue_size =1)
        
        self.joint1_msg = Twist()
        self.joint2_msg = Twist() 
        self.real_robot_msg = Int32()
        self.avanza = 0 
        self.gira = 0
        self.new_angle = False
        
        #'''
        subscriber = rospy.Subscriber("/gazebo/model_states",ModelStates,callback=self.gazebo_callback)
        self.subscriber = rospy.Subscriber("/segway/joint_states",JointState,callback=self.robot_configuration_callback)
        self.subscriber = rospy.Subscriber("/imu",Imu,callback=self.imu_callback)
        self.teleop = rospy.Subscriber("/turtle1/cmd_vel", Twist ,callback=self.teleop_callback)
        self.subscriber = rospy.Subscriber("/segzay/laser/scan", LaserScan, callback=self.laser_callback)
        self.reset = rospy.ServiceProxy("/gazebo/reset_world",Empty)
        self.pause = rospy.ServiceProxy("/gazebo/pause_physics",Empty)
        self.unpause = rospy.ServiceProxy("/gazebo/unpause_physics",Empty)
        #'''
        
        #self.subscriber = rospy.Subscriber("/angle",Float32,callback=self.mpu5060_callback)        
        self.distance = 0
        self.tilt = 0
        self.postion = 0
        self.q = []
        self.roll = 0
        self.pitch= 0 
        self.yaw = 0 

        self.robot_configuration = []

    def laser_callback(self,data):
        self.distance = data.ranges[0]
        
    def reset_pose(self):
            Robot.joint1_msg.angular.z = 0
            Robot.joint2_msg.angular.z = 0
            
            Robot.joint1.publish(Robot.joint1_msg)
            Robot.joint2.publish(Robot.joint2_msg)           
    
    def teleop_callback(self,data):
        self.avanza += data.linear.x* 0.01
        self.gira = data.angular.z
    def mpu5060_callback(self, date):
        self.new_angle = True
        self.roll = date.data * 0.01745329
        
    def imu_callback(self, date):
        self.q = [
            date.orientation.x,
            date.orientation.y,
            date.orientation.z,
            date.orientation.w]
        x, y, z, w = self.q
        t0 = +2.0 * (w * x + y * z)
        t1 = +1.0 - 2.0 * (x * x + y * y)
        self.roll = math.atan2(t0, t1)
        t2 = +2.0 * (w * y - z * x)  
        t2 = +1.0 if t2 > +1.0 else t2
        t2 = -1.0 if t2 < -1.0 else t2
        self.pitch = math.asin(t2)
        t3 = +2.0 * (w * z + x * y)
        t4 = +1.0 - 2.0 * (y * y + z * z)
        self.yaw = math.atan2(t3, t4)
    
    
    def robot_configuration_callback(self, data):
        self.robot_configuration = data.position
        
    def gazebo_callback(self, data):
        self.position = data.pose[1].position.y
        

In [5]:
class Memory:
    def __init__(self):
        self.actions = []
        self.states = []
        self.logprobs = []
        self.rewards = []
    
    def clear_memory(self):
        del self.actions[:]
        del self.states[:]
        del self.logprobs[:]
        del self.rewards[:]

class ActorCritic(nn.Module):
    def __init__(self, state_dim, action_dim, n_var, action_std):
        super(ActorCritic, self).__init__()
        # action mean range -1 to 1
        self.actor =  nn.Sequential(
                nn.Linear(state_dim, n_var),
                nn.Tanh(),
                nn.Linear(n_var, n_var),
                nn.Tanh(),
                nn.Linear(n_var, action_dim),
                nn.Tanh()
                )
        # critic
        self.critic = nn.Sequential(
                nn.Linear(state_dim, n_var),
                nn.Tanh(),
                nn.Linear(n_var, n_var),
                nn.Tanh(),
                nn.Linear(n_var, 1)
                )
        self.action_var = torch.full((action_dim,), action_std*action_std).to(device)
        
    def forward(self):
        raise NotImplementedError
    
    def act(self, state, memory):
        action_mean = self.actor(state)
        dist = MultivariateNormal(action_mean, torch.diag(self.action_var).to(device))
        action = dist.sample()
        action_logprob = dist.log_prob(action)
        
        memory.states.append(state)
        memory.actions.append(action)
        memory.logprobs.append(action_logprob)
        
        return action.detach()
    
    def evaluate(self, state, action):
        action_mean = self.actor(state)
        dist = MultivariateNormal(torch.squeeze(action_mean), torch.diag(self.action_var))
        
        action_logprobs = dist.log_prob(torch.squeeze(action))
        dist_entropy = dist.entropy()
        state_value = self.critic(state)
        
        return action_logprobs, torch.squeeze(state_value), dist_entropy

In [6]:
class PPO:
    def __init__(self, state_dim, action_dim, n_latent_var, action_std, lr, betas, gamma, K_epochs, eps_clip):
        self.lr = lr
        self.betas = betas
        self.gamma = gamma
        self.eps_clip = eps_clip
        self.K_epochs = K_epochs
        
        self.policy = ActorCritic(state_dim, action_dim, n_latent_var, action_std).to(device)
        self.optimizer = torch.optim.Adam(self.policy.parameters(),
                                              lr=lr, betas=betas)
        self.policy_old = ActorCritic(state_dim, action_dim, n_latent_var, action_std).to(device)
        
        self.MseLoss = nn.MSELoss()
    
    def select_action(self, state, memory):
        state = torch.FloatTensor(state.reshape(1, -1)).to(device)
        return self.policy_old.act(state, memory).cpu().data.numpy().flatten()
    
    def update(self, memory):
        print ("Updating policy...")
        # Monte Carlo estimate of rewards:
        rewards = []
        discounted_reward = 0
        for reward in reversed(memory.rewards):
            discounted_reward = reward + (self.gamma * discounted_reward)
            rewards.insert(0, discounted_reward)
        
        # Normalizing the rewards:
        rewards = torch.tensor(rewards).to(device)
        rewards = (rewards - rewards.mean()) / (rewards.std() + 1e-5)
        
        # convert list to tensor
        old_states = torch.stack(memory.states).to(device).detach()
        old_actions = torch.stack(memory.actions).to(device).detach()
        old_logprobs = torch.squeeze(torch.stack(memory.logprobs)).to(device).detach()

        
        # Optimize policy for K epochs:
        for _ in range(self.K_epochs):
            # Evaluating old actions and values :
            logprobs, state_values, dist_entropy = self.policy.evaluate(old_states, old_actions)
     
            # Finding the ratio (pi_theta / pi_theta__old):
            ratios = torch.exp(logprobs - old_logprobs.detach())

            # Finding Surrogate Loss:
            advantages = rewards - state_values.detach()
            surr1 = ratios * advantages
            surr2 = torch.clamp(ratios, 1-self.eps_clip, 1+self.eps_clip) * advantages
            loss = -torch.min(surr1, surr2) + 0.5*self.MseLoss(state_values, rewards) - 0.01*dist_entropy
            
            # take gradient step
            self.optimizer.zero_grad()
            loss.mean().backward()
            self.optimizer.step()
            
        # Copy new weights into old policy:
        self.policy_old.load_state_dict(self.policy.state_dict())
        
    def load(self, path): 
        checkpoint = torch.load(path)
        ppo.policy_old.load_state_dict(checkpoint['old_policy_state_dict'])
        ppo.policy.load_state_dict(checkpoint['policy_state_dict'])
        ppo.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

        ppo.policy_old.train()
        ppo.policy.train()
        
    def save(self,path):
        torch.save({
            'old_policy_state_dict': self.policy_old.state_dict(),
            'policy_state_dict': self.policy.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict()
            }, path)
        

In [7]:
    env_name = "LunarLanderContinuous-v2"
    render = False
    solved_reward = 200         # stop training if avg_reward > solved_reward
    log_interval = 20           # print avg reward in the interval
    max_episodes = 10000000      # max training episodes
    max_timesteps = 10000         # max timesteps in one episode
    n_latent_var = 64           # number of variables in hidden layer
    update_timestep = 50000     # update policy every n timesteps
    action_std = 0.1#5            # constant std for action distribution
    lr = 0.002
    betas = (0.9, 0.999)
    gamma = 0.99                # discount factor
    K_epochs = 5                # update policy for K epochs
    eps_clip = 0.2             # clip parameter for PPO
    random_seed = None

In [8]:
rospy.init_node('NN_ROBOT1')

In [9]:
Robot = NN_ROBOT1()

In [10]:
#Robot.reset()

In [11]:
state_dim = 3
action_dim = 2

In [12]:
PATH = '/home/giuseppe/catkin_ws/src/self_balancing_ai/notebook/dc_motor_temp.tar'
memory = Memory()
ppo = PPO(state_dim, action_dim, n_latent_var, action_std, lr, betas, gamma, K_epochs, eps_clip)  
rate = rospy.Rate(10) # 10hz
ppo.load(PATH)

In [13]:
running_reward = 0
avg_length = 0
time_step = 0
fine = False
best_reward = 0 
# training loop
best_reward_cnt = 0 
old_roll = 0
reward_print = 0
old_running_reward = 600
Robot.reset()
Robot.reset_pose()
reward_graph = 0
for i_episode in range(1, max_episodes+1):
    Robot.reset()
    Robot.reset_pose()
    start_time = time.time()
    avg_length = 0       
    old_pos = 0
    old_roll= 0
    if running_reward > (best_reward + 5):
        best_reward_cnt += 1
        best_reward = running_reward
        
    running_reward = 0
    motor_position  = 0
    for t in range(max_timesteps):
        reward_graph += 1
        '''
        while(not Robot.new_angle):
            continue
        Robot.new_angle = False
        '''
        time_step +=1
        Ts = time.time() - start_time
        start_time = time.time()
        vel = (Robot.roll - old_roll)/Ts
        old_roll = Robot.roll
        state = [Robot.roll, vel, 0]
   
        # Running policy_old:
        action = ppo.select_action(np.array(state), memory)[0] * 10
       
        Robot.joint1_msg.angular.z = action 
        Robot.joint2_msg.angular.z = action
        Robot.joint2.publish(Robot.joint1_msg)
        Robot.joint1.publish(Robot.joint2_msg) 
        time.sleep(0.02)                  #ritardo di movimento 
        
        reward =  1 - abs(Robot.roll)    #reward for stay in a vertical position
            
        reward_print += reward
        #print ("Roll: {}\tReaward: {}".format(Robot.roll, reward))
        if (abs(Robot.roll) > 0.8):
            reward = - 100
            reward_print += reward
            fine = True
        # Saving reward:
        memory.rewards.append(reward)
        
            
        # update if its time
        if time_step % update_timestep == 0:
            ppo.update(memory)
            memory.clear_memory()
            time_step = 0
        running_reward += reward
        if (fine):
            fine = False
            writer.add_scalar("Reward", running_reward, i_episode)
            #print ("Reward episode: {}".format(running_reward))
            if running_reward > old_running_reward: 
                ppo.save(PATH) 
                #print("Old reward: {}".format(old_running_reward))
                old_running_reward = running_reward
                print("Saved")
            reward_print = 0
            break
    

Updating policy...
Updating policy...
Updating policy...
Updating policy...


KeyboardInterrupt: 

In [14]:
memory.clear_memory()

In [15]:
PATH = '/home/giuseppe/catkin_ws/src/self_balancing_ai/notebook/dc_motor_temp.tar'

In [16]:
ppo.save(PATH)

In [17]:
old_running_reward

600

In [18]:
i_episode

4541

In [ ]:
l = 8 
for l in range(8,10):
    print (l)